In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd

import os
import time
import gc

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.ensemble import ExtraTreesClassifier

from lightgbm import LGBMClassifier

import random

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

Mounted at /content/drive


In [ ]:
import pickle
from sklearn.model_selection import StratifiedKFold

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
def train_model(data_loader, model, optimizer, device):
  """
  This function does training for one epoch
  :param data_loader: this is the pytorch dataloader
  :param model: pytorch model
  :param optimizer: optimizer, for e.g. adam, sgd, etc
  :param device: cuda/cpu
  """
  # put the model in train mode
  model.train()
  # scheduler = get_scheduler(optimizer, 'CosineAnnealingWarmRestarts')
  # go over every batch of data in data loader
  for data in tqdm(data_loader):
    # remember, we have image and targets
    # in our dataset class
    inputs = data["X"]
    targets = data["targets"]

    # move inputs/targets to cuda/cpu device
    inputs = inputs.to(device, dtype=torch.float)
    targets = targets.to(device, dtype=torch.float)

    # zero grad the optimizer
    optimizer.zero_grad()
    #do the forward step of model
    outputs = model(inputs)
    # calculate loss

    loss = nn.CrossEntropyLoss()(outputs,targets.long())
    # backward step the loss
    loss.backward()
    # step optimizer
    optimizer.step()
    # if you have a scheduler, you either need to
    # step it here or you have to step it after
    # the epoch. here, we are not using any learning
    # rate scheduler
  # scheduler.step()

def evaluate_model(data_loader, model, device):
  """
  This function does evaluation for one epoch
  :param data_loader: this is the pytorch dataloader
  :param model: pytorch model
  :param device: cuda/cpu
  """
  # put model in evaluation mode
  model.eval()

  # init lists to store targets and outputs
  final_targets = []
  final_outputs = []

  # we use no_grad context
  with torch.no_grad():
    for data in tqdm(data_loader):
      inputs = data["X"]
      targets = data["targets"]
      inputs = inputs.to(device, dtype=torch.float)
      targets = targets.to(device, dtype=torch.float)

      # do the forward step to generate prediction
      output = model(inputs)

      # convert targets and outputs to lists
      targets = targets.detach().cpu().numpy().tolist()
      output = output.detach().cpu().numpy().tolist()

      # extend the original list
      final_targets.extend(targets)
      final_outputs.extend(output)

  # return final output and final targets
  return final_outputs, final_targets

import os
def save_checkpoint(model, optimizer, path):
    if not os.path.exists(os.path.dirname(path)):
        print("Creating directories on path: `{}`".format(path))
        os.makedirs(os.path.dirname(path))

    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    }, path)


def load_checkpoint(model, path):
    checkpoint = torch.load(path)

    model.load_state_dict(checkpoint["model_state_dict"])

    optimizer = torch.optim.Adam(model.parameters())
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    return model, optimizer


def save_model(model, path):
  if not os.path.exists(os.path.dirname(path)):
      print("Creating directories on path: `{}`".format(path))
      os.makedirs(os.path.dirname(path))

  torch.save({
      "model_state_dict": model.state_dict(),
  }, path)


def load_model(model, path):
  restore_dict = torch.load(path)

  model.load_state_dict(restore_dict["model_state_dict"])
  model.eval()

  return model

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

def get_scheduler(optimizer, scheduler):
    if scheduler=='ReduceLROnPlateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=4, verbose=True, eps=1e-6)
    elif scheduler=='CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6, last_epoch=-1)
    elif scheduler=='CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
    return scheduler

In [ ]:
LSTM_UNITS = 128
GRU_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 100

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_shape, num_aux_targets):
        super(NeuralNet, self).__init__()
        self.lstm = nn.LSTM(input_shape, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(input_shape, GRU_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(LSTM_UNITS*2, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(GRU_UNITS*2, DENSE_HIDDEN_UNITS)

        self.attention = nn.Sequential(
            nn.Linear(DENSE_HIDDEN_UNITS, 256),
            nn.Tanh(),
            nn.Linear(256, 1),
            nn.Softmax(dim=1)
        )


        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        # self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)

    def forward(self, x):
        x = x.view((-1,1,100))
        h_lstm, _ = self.lstm(x)
        h_gru, _ = self.gru(x)

        h_lstm = h_lstm.squeeze()
        h_gru = h_gru.squeeze()

        # global average pooling
        # avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        # max_pool, _ = torch.max(h_lstm2, 1)

        # h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_lstm))
        h_conc_linear2  = F.relu(self.linear2(h_gru))

        hidden = (h_conc_linear1 + h_conc_linear2)/2

        weights = self.attention(hidden)

        context_vector = weights * hidden

        # print(context_vector.shape)
        # print(weights.shape)
        # print(hidden.shape)

        results = self.linear_out(context_vector)

        return nn.Softmax(dim=1)(results)

In [ ]:
models_nn = [
             '/content/drive/MyDrive/models/model_fold0_33.pth',
             '/content/drive/MyDrive/models/model_fold1_48.pth',
             '/content/drive/MyDrive/models/model_fold2_49.pth',
             '/content/drive/MyDrive/models/model_fold3_39.pth',
             '/content/drive/MyDrive/models/model_fold4_38.pth'

]
models_xgb = [
              '/content/drive/MyDrive/models/xgb_fold0.b5',
              '/content/drive/MyDrive/models/xgb_fold1.b5',
              '/content/drive/MyDrive/models/xgb_fold2.b5',
              '/content/drive/MyDrive/models/xgb_fold3.b5',
              '/content/drive/MyDrive/models/xgb_fold4.b5'
]

models_lgbm = [
               '/content/drive/MyDrive/models/lgbm_fold0.b5',
               '/content/drive/MyDrive/models/lgbm_fold1.b5',
               '/content/drive/MyDrive/models/lgbm_fold2.b5',
               '/content/drive/MyDrive/models/lgbm_fold3.b5',
               '/content/drive/MyDrive/models/lgbm_fold4.b5'
]

In [ ]:
import torch

trainable_params = 0


for model_nn, model_lgbm, model_xgb in zip(models_nn, models_lgbm, models_xgb):
  model = NeuralNet(100, 9)
  model, _ = load_checkpoint(model, model_nn)
  model_xgb = pickle.load(open(model_xgb, 'rb'))
  model_lgbm = pickle.load(open(model_lgbm, 'rb'))
  trainable_params += sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params/5}")


Number of trainable parameters: 811530.0
